# Inferencia de las fuerzas moleculares de los elementos del dataset QM9

## Importación de módulos

In [1]:
import torch
import numpy as np
import pandas as pd
import os
from scripts.qm9 import QM9

/root/mambaforge/envs/torchmd-net/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Definición de directorios

In [2]:
path_splits_npy = "/workspace/tesis-potenciales-moleculares/inferencia-fuerzas/splits.npz"

# Ruta al dataset que uso torchmd-net
path_torchmdnet_dataset = "/workspace/tesis-potenciales-moleculares/inferencia-fuerzas/dataset_qm9.pt"

# Ruta al modelo entrenado
path_trained_model="/workspace/tesis-potenciales-moleculares/inferencia-fuerzas/epoch=669-val_loss=0.0003-test_loss=0.0064.ckpt"

## Cargar modelo

In [3]:
from torchmdnet.models.model import load_model
model= load_model(path_trained_model, derivative=True)

## Cargar database

In [4]:
# Cargar archivo del dataset que utiliza torchmd-net
torchmdnet_dataset = torch.load(path_torchmdnet_dataset)

## Inferencias individuales

In [5]:
#número de indice de molecula
num_mol=10

In [6]:
molecula=torchmdnet_dataset[num_mol]

z_sample = molecula.z
pos_sample = molecula.pos
n_energy_sample = molecula.y
# Hacer la inferencia
n_energy_inferred, n_forces_inferred = model(z_sample, pos_sample)    

In [7]:
print(len(z_sample))
z_sample=z_sample.tolist()
z_sample=" ".join(map(str, z_sample))
print(z_sample)

7
6 6 8 1 1 1 1


In [8]:
pos_sample=pos_sample.flatten().tolist()
pos_sample=" ".join(map(str, pos_sample))
print(pos_sample)

-0.002899999963119626 1.5098999738693237 0.008700000122189522 0.026100000366568565 0.0032999999821186066 -0.03750000149011612 0.942300021648407 -0.6550999879837036 -0.45680001378059387 0.9228000044822693 1.926300048828125 -0.39149999618530273 -0.8619999885559082 1.878499984741211 -0.5648000240325928 -0.15049999952316284 1.8438999652862549 1.0428999662399292 -0.8944000005722046 -0.4864000082015991 0.357699990272522


In [9]:
n_energy_sample=float(n_energy_sample)
print(n_energy_sample)

-4184.77392578125


In [10]:
n_energy_inferred=float(n_energy_inferred)
print(n_energy_inferred) 

-4184.77587890625


In [11]:
n_forces_inferred=n_forces_inferred.flatten().tolist()
n_forces_inferred=" ".join(map(str, n_forces_inferred))
print(n_forces_inferred)

-0.20835840702056885 -0.7185043096542358 0.07556511461734772 0.09281401336193085 1.2005568742752075 -0.013574824668467045 0.34101295471191406 -0.08474382758140564 -0.15249665081501007 -0.060598645359277725 -0.16454747319221497 0.02326197549700737 0.14518292248249054 -0.23716624081134796 0.09191824495792389 0.025890395045280457 -0.23225143551826477 -0.17816713452339172 -0.3359432518482208 0.23665641248226166 0.15349328517913818


## Creación de un nuevo database con fuerzas inferidas

## Importación de módulos

In [12]:
import numpy as np
import h5py
import random
import os
import shutil

## Creación de carpetas

In [13]:
nombre_carpeta = "temp"

if os.path.exists(nombre_carpeta) and os.path.isdir(nombre_carpeta):
    shutil.rmtree(nombre_carpeta)
    os.mkdir(nombre_carpeta)
else:
    os.mkdir(nombre_carpeta)

nombre_carpeta = "datos-procesados"

if os.path.exists(nombre_carpeta) and os.path.isdir(nombre_carpeta):
    shutil.rmtree(nombre_carpeta)
    os.mkdir(nombre_carpeta)
else:
    os.mkdir(nombre_carpeta)

In [14]:
for molecula in torchmdnet_dataset:
    #Definición de variables
    z_sample = molecula.z
    pos_sample = molecula.pos
    n_energy_sample = molecula.y

    # Realización de la inferencia
    n_energy_inferred, n_forces_inferred = model(z_sample, pos_sample)

    # Transformar la inferencia en texto
    z_sample=z_sample.tolist()
    z_sample=" ".join(map(str, z_sample))

    pos_sample=pos_sample.flatten().tolist()
    pos_sample=" ".join(map(str, pos_sample))

    n_energy_sample=float(n_energy_sample)

    n_energy_inferred=float(n_energy_inferred)

    n_forces_inferred=n_forces_inferred.flatten().tolist()
    n_forces_inferred=" ".join(map(str, n_forces_inferred))

    # Guardar en archivos temporales
    with open("temp/z.data", "a") as file:
        line = z_sample
        file.write(line + "\n")
        
    with open("temp/pos.data", "a") as file:
        line = pos_sample
        file.write(line + "\n")

    with open("temp/energy_sample.data", "a") as file:
        line = str(n_energy_sample)
        file.write(line + "\n")

    with open("temp/energy_inferred.data", "a") as file:
        line = str(n_energy_inferred)
        file.write(line + "\n")

    with open("temp/forces_inferred.data", "a") as file:
        line = n_forces_inferred
        file.write(line + "\n")